In [1]:
import scrapy
from selenium import webdriver
import pandas
from urllib.parse import urlparse
from scrapy.crawler import CrawlerProcess


In [2]:
name = "salarios"
siape = dict()
urls = list()
arq = pandas.read_csv("servidores_ativos_ufpel.csv")
servidore = arq["nome"]
siapeUtil = arq["siape"]
main_site = "https://www.portaldatransparencia.gov.br/servidores/Servidor-ListaServidores.asp?bogus=1&Pagina=1&TextoPesquisa="

In [3]:
servidore

0            MARILIA GUIDOTTI CORREA
1          Rosiane Schwantz do Couto
2       HELEN LETICIA GRALA JACOBSEN
3              FLORIBALDO DIAS PERES
4                  LUIZ PAULO DUARTE
                    ...             
2671     CARLOS ARTHUR SALDANHA DIAS
2672        CAROLINA ANDERSSON BUNDE
2673      LISANDRA MACHADO KERCHIRNE
2674          RENATA PEREIRA CARDOSO
2675          VITOR HUGO KARST FELIX
Name: nome, Length: 2676, dtype: object

In [4]:
siapeUtil

0       2103752
1       1844853
2       1854388
3        420664
4        421000
         ...   
2671    3558505
2672    2354376
2673    1894217
2674    2183279
2675    2346070
Name: siape, Length: 2676, dtype: int64

In [5]:
arq

,cod_unidade,unidade_lotacao,siape,nome,regime_trabalho,titulacao,categoria,cargo,Unnamed: 8,Unnamed: 9
0,38.0,Agência de Desenv. da Bacia da Lagoa Mirim,2103752,MARILIA GUIDOTTI CORREA,40h,Especialização NS,Técnico administrativo,Técnico de Laboratório,NaN,NaN
1,38.0,Agência de Desenv. da Bacia da Lagoa Mirim,1844853,Rosiane Schwantz do Couto,40h,Mestrado,Técnico administrativo,Assistente em Administração,NaN,NaN
2,40.0,Auditoria Interna,1854388,HELEN LETICIA GRALA JACOBSEN,40h,Mestrado,Técnico administrativo,Auditor,NaN,NaN
3,36.0,Biotério Central,420664,FLORIBALDO DIAS PERES,40h,NaN,Técnico administrativo,Mestre de Edificações e Infraestrutura,NaN,NaN
4,36.0,Biotério Central,421000,LUIZ PAULO DUARTE,40h,NaN,Técnico administrativo,Assistente em Administração,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2671,NaN,NaN,3558505,CARLOS ARTHUR SALDANHA DIAS,40h,NaN,Técnico administrativo,Auditor,NaN,NaN
2672,NaN,NaN,2354376,CAROLINA ANDERSSON BUNDE,40h,Especialização NS,Técnico administrativo,Assistente Social,NaN,NaN
2673,NaN,NaN,1894217,LISANDRA MACHADO KERCHIRNE,40h,Especialização NS,Técnico administrativo,Assistente em Administração,NaN,NaN
2674,NaN,NaN,2183279,RENATA PEREIRA CARDOSO,40h,Mestrado,Técnico administrativo,Auditor,NaN,NaN


In [6]:
arq = arq.drop(columns=['Unnamed: 8','Unnamed: 9','cod_unidade'],axis=1)

In [7]:
def mount_link(name):
    main_site = "https://www.portaldatransparencia.gov.br/servidores/consulta?paginacaoSimples=true&tamanhoPagina=&offset=&direcaoOrdenacao=asc&colunasSelecionadas=detalhar%2Ctipo%2Ccpf%2Cnome%2CorgaoServidorExercicio%2CorgaoServidorLotacao%2Cmatricula%2CtipoVinculo%2Cfuncao&palavraChave="+name.replace(' ', '+')
    return urlparse(main_site).geturl()

In [20]:
print(mount_link("MARILIA GUIDOTTI CORREA"))

https://www.portaldatransparencia.gov.br/servidores/consulta?paginacaoSimples=true&tamanhoPagina=&offset=&direcaoOrdenacao=asc&colunasSelecionadas=detalhar%2Ctipo%2Ccpf%2Cnome%2CorgaoServidorExercicio%2CorgaoServidorLotacao%2Cmatricula%2CtipoVinculo%2Cfuncao&palavraChave=MARILIA+GUIDOTTI+CORREA


In [21]:
arq["link"] = arq["nome"].apply(lambda row: mount_link(row))

In [22]:
arq['link']

0       https://www.portaldatransparencia.gov.br/servi...
1       https://www.portaldatransparencia.gov.br/servi...
2       https://www.portaldatransparencia.gov.br/servi...
3       https://www.portaldatransparencia.gov.br/servi...
4       https://www.portaldatransparencia.gov.br/servi...
                              ...                        
2671    https://www.portaldatransparencia.gov.br/servi...
2672    https://www.portaldatransparencia.gov.br/servi...
2673    https://www.portaldatransparencia.gov.br/servi...
2674    https://www.portaldatransparencia.gov.br/servi...
2675    https://www.portaldatransparencia.gov.br/servi...
Name: link, Length: 2676, dtype: object

In [26]:
arq['link'][0]

'https://www.portaldatransparencia.gov.br/servidores/consulta?paginacaoSimples=true&tamanhoPagina=&offset=&direcaoOrdenacao=asc&colunasSelecionadas=detalhar%2Ctipo%2Ccpf%2Cnome%2CorgaoServidorExercicio%2CorgaoServidorLotacao%2Cmatricula%2CtipoVinculo%2Cfuncao&palavraChave=MARILIA+GUIDOTTI+CORREA'

In [27]:

class salary_spider(scrapy.Spider):
    name = "salary_spider"
    start_urls = arq['link'][0]
    
    def parse(self, response):
        for quote in response.xpath('//div[2]/section[1]/div[1]/div[2]/span/a'):
            yield {'quote': quote.extract()}

process = CrawlerProcess()


2021-06-04 21:35:05 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-06-04 21:35:05 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.5 (default, May 27 2021, 13:30:53) - [GCC 9.3.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-glibc2.29
2021-06-04 21:35:05 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor


In [29]:
process.crawl(salary_spider)
process.start()

2021-06-04 21:35:20 [scrapy.crawler] INFO: Overridden settings:
{}
2021-06-04 21:35:20 [scrapy.extensions.telnet] INFO: Telnet Password: fbeae53efef56b51
2021-06-04 21:35:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-06-04 21:35:20 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrap